In [2]:
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=cf6fe0dde49f6c6f9d1169872ef3c872e41a55d452ea138fb5b5d5e75e7acd9f
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [3]:
import random
import sys
import os
from nltk.translate.bleu_score import sentence_bleu
import json
import codecs
json_load = lambda x: json.load(codecs.open(x, 'r', encoding='utf-8'))
json_dump = lambda d, p: json.dump(d, codecs.open(p, 'w', 'utf-8'), indent=2, ensure_ascii=False)
import numpy as np
from nltk import word_tokenize
import nltk
nltk.download('punkt')
from rouge_score import rouge_scorer
from scipy import interpolate
from pathlib import Path
from typing import List, Union, Iterable
from itertools import zip_longest
from collections import defaultdict
import numpy as np

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
rouge_map = {
    "rouge1": [{"length":3.9826927517,"score":0.0682902912},
{"length":5.9549255862,"score":0.0978827539},
{"length":7.9076051135,"score":0.1224228823},
{"length":9.81988077,"score":0.1420500099},
{"length":11.6670102227,"score":0.15667295},
{"length":13.4282499633,"score":0.1688965384},
{"length":15.0784735196,"score":0.1782539613},
{"length":16.6026847751,"score":0.1855030362},
{"length":18.0075526354,"score":0.1913154379},
{"length":19.2746489221,"score":0.1960990131},
{"length":20.4229391885,"score":0.1993243852},
{"length":21.4450618191,"score":0.2024686698},
{"length":22.3082453452,"score":0.204501986},
{"length":23.1111106446,"score":0.2062906734},
{"length":28.2587921661,"score":0.2123504881},
{"length":55.718749344,"score":0.2515012516},
{"length":81.5821532778,"score":0.2560368626},
{"length":105.2805852348,"score":0.2501756483},
{"length":126.637654023,"score":0.2424150409},
{"length":145.7091626608,"score":0.2348272003},
{"length":162.4257730011,"score":0.2280278165},
{"length":177.1731711413,"score":0.2224555269},
{"length":190.1122798547,"score":0.2179352276},
{"length":201.4505573165,"score":0.2142724833}],
    "rouge2":[{"length":3.9826927517,"score":0.0144407758},
{"length":5.9549255862,"score":0.0249529835},
{"length":7.9076051135,"score":0.0347267516},
{"length":9.81988077,"score":0.0427076786},
{"length":11.6670102227,"score":0.0485375859},
{"length":13.4282499633,"score":0.0535693814},
{"length":15.0784735196,"score":0.0572324993},
{"length":16.6026847751,"score":0.0602716788},
{"length":18.0075526354,"score":0.0628290212},
{"length":19.2746489221,"score":0.0649507089},
{"length":20.4229391885,"score":0.0661721408},
{"length":21.4450618191,"score":0.0678942278},
{"length":22.3082453452,"score":0.0688855871},
{"length":23.1111106446,"score":0.0697352193},
{"length":28.2587921661,"score":0.0736741358},
{"length":55.718749344,"score":0.0946583707},
{"length":81.5821532778,"score":0.1034647258},
{"length":105.2805852348,"score":0.1066520132},
{"length":126.637654023,"score":0.1083127966},
{"length":145.7091626608,"score":0.1088493736},
{"length":162.4257730011,"score":0.1086376524},
{"length":177.1731711413,"score":0.1083631105},
{"length":190.1122798547,"score":0.1080597696},
{"length":201.4505573165,"score":0.1077033711}],
"rougeL":[{"length":3.9826927517,"score":0.0638302511},
{"length":5.9549255862,"score":0.0885365378},
{"length":7.9076051135,"score":0.1080729605},
{"length":9.81988077,"score":0.1226784665},
{"length":11.6670102227,"score":0.13304084},
{"length":13.4282499633,"score":0.1412546317},
{"length":15.0784735196,"score":0.1473504772},
{"length":16.6026847751,"score":0.1517939352},
{"length":18.0075526354,"score":0.1552496947},
{"length":19.2746489221,"score":0.1581296506},
{"length":20.4229391885,"score":0.1597121746},
{"length":21.4450618191,"score":0.1616429679},
{"length":22.3082453452,"score":0.162688815},
{"length":23.1111106446,"score":0.163630746},
{"length":28.2587921661,"score":0.1666532163},
{"length":55.718749344,"score":0.1846472648},
{"length":81.5821532778,"score":0.1836643033},
{"length":105.2805852348,"score":0.1785305626},
{"length":126.637654023,"score":0.1737320905},
{"length":145.7091626608,"score":0.1693876728},
{"length":162.4257730011,"score":0.1657265038},
{"length":177.1731711413,"score":0.162906589},
{"length":190.1122798547,"score":0.1606680241},
{"length":201.4505573165,"score":0.1588698335}]
}

In [5]:
class ScicapSummarizationEval():
    def __init__(self):
        self.metric_list = ["rouge1", "rouge2", "rougeL"]
        self.rouge = rouge_scorer.RougeScorer(
            self.metric_list,
            use_stemmer=True,
        )

        # build score normalization
        
        self.normalization_funcs = {}
        for metric in self.metric_list:
            random_data = rouge_map[metric]
            length_list = np.array([d["length"] for d in random_data])
            score_list = np.array([d["score"] for d in random_data])
            normalization_func = interpolate.interp1d(
                length_list,
                score_list,
                bounds_error=False,
                fill_value="extrapolate",
            )
            self.normalization_funcs[metric] = normalization_func
        
    def clean_text(self, text):
        return text.lower()

    def evaluate(
        self, 
        hypothesis: List[str],
        reference: List[str],
    ):
        assert len(hypothesis) == len(reference), "The number of hypothesis and reference must be the same."

        score_list = []
        for hyp, ref in zip(hypothesis, reference):
            hyp = self.clean_text(hyp)
            ref = self.clean_text(ref)

            # compute scores and match
            scores = self.rouge.score(ref, hyp)
            bleu = sentence_bleu([ref.split()], hyp.split())
            score_dict = {
                "rouge1": scores["rouge1"].fmeasure,
                "rouge2": scores["rouge2"].fmeasure,
                "rougeL": scores["rougeL"].fmeasure,
                "length": len(word_tokenize(hyp)),
                "bleu-4": bleu
            }
            score_list.append(score_dict)
        
        # compute mean and normalize scores
        score_dict = {
            "length": np.mean([d["length"] for d in score_list]),
        }
        for metric in self.metric_list:
            score_dict[metric] = np.mean([d[metric] for d in score_list])
            score_dict[f"{metric}_normalized"] = score_dict[metric] / self.normalization_funcs[metric](score_dict["length"])
        score_dict["bleu-4"] = np.mean([d["bleu-4"] for d in score_list])
        return score_dict
        
def evaluate(test_annotation_file, user_submission_file, **kwargs):
    print("Starting Evaluation.....")
    #for metric, score in cocoEval.eval.items():
    #    print(metric, score)

    # load data
    ground_truth = json_load(test_annotation_file)
    submission = json_load(user_submission_file)
    
    print("align annotations and submission file..")
    
    
    # align data
    data = {sample["image_id"]: sample for sample in ground_truth["annotations"]}
    for prediction in submission:
        image_id = prediction["image_id"]
        data[image_id]["prediction"] = prediction["caption"]

    # convert to list
    hypothesis = []
    reference = []
    print(data)
    for image_id, sample in data.items():
        try:
            hypothesis.append(sample["prediction"])
            reference.append(sample["caption"])
        except:
          print("Missing id: ", image_id)
          return
          
    assert len(ground_truth['images']) == len(submission), "The number of submission samples must be the same as the annotated samples."
    # evaluate
    evaluator = ScicapSummarizationEval()
    score = evaluator.evaluate(
        hypothesis=hypothesis,
        reference=reference,
    )

    output = {}
    output["result"] = [
        {
            "test_split": {
                "BLEU-4": score['bleu-4'],
                "Rouge-1": score['rouge1'],
                "Rouge-1-normalized": score['rouge1_normalized'],
                "Rouge-2": score['rouge2'],
                "Rouge-2-normalized": score['rouge2_normalized'],
                "Rouge-L": score['rougeL'],
                "Rouge-L-normalized": score['rougeL_normalized'],
            }
        }
    ]
    # To display the results in the result file
    output["submission_result"] = output["result"][0]
    print("Completed evaluation!")
    return output

In [6]:
# This is a sample for the evaluation, please replace the files with your own prediction and the corresponding data split
evaluate("/content/test_annotations_devsplit.json", "/content/submission_dev.json")

Starting Evaluation.....
align annotations and submission file..
{7424363: {'image_id': 7424363, 'id': 8492163, 'caption': 'Fig. 1. The coefficient of Pe2 in the small-Pe optimal enhancement (32) with n = 1 (first zero). The optimal enhancement is achieved for m = 2, then drops off slowly.', 'caption_no_index': 'The coefficient of Pe2 in the small-Pe optimal enhancement (32) with n =  (first zero). The optimal enhancement is achieved for m = 2, then drops off slowly.', 'paragraph': ['Since j m,n increases monotonically with n, we must take n = 1 to minimize T . Thus the optimal streamlines pattern displays a single cell in the radial direction. It is then a simple matter of enumerating the zeros j m,1 to find that the integrated mean exit time is minimized for (m, n) = (2, 1), independent of the Péclet number (see Figure 1). The streamline and mean exit time patterns are illustrated in Figure 2.'], 'mention': [['It is then a simple matter of enumerating the zeros j m,1 to find that the

{'result': [{'test_split': {'BLEU-4': 0.25042009669367926,
    'Rouge-1': 0.5714285714285715,
    'Rouge-1-normalized': 3.213721570521792,
    'Rouge-2': 0.5499999999999999,
    'Rouge-2-normalized': 9.639262673254814,
    'Rouge-L': 0.5714285714285715,
    'Rouge-L-normalized': 3.8856673511349342}}],
 'submission_result': {'test_split': {'BLEU-4': 0.25042009669367926,
   'Rouge-1': 0.5714285714285715,
   'Rouge-1-normalized': 3.213721570521792,
   'Rouge-2': 0.5499999999999999,
   'Rouge-2-normalized': 9.639262673254814,
   'Rouge-L': 0.5714285714285715,
   'Rouge-L-normalized': 3.8856673511349342}}}